# InfluxDB

In [32]:
from influxdb_client   import InfluxDBClient, BucketRetentionRules,WritePrecision
import influxdb_client

In [33]:
import random

In [34]:
import time

In [35]:
import pandas as pd

## Create a Client Object

In [36]:
client = InfluxDBClient(url="http://localhost:8086", token= "my_password", org="primary")

In [37]:
def myprint(table):
    for table in tables:
        print(table)
        for row in table.records:
            print (row.values)

# Bucket

In [38]:
bucket_api=client.buckets_api()

In [39]:
bucket_api.create_bucket(bucket_name="myFirstBucket",org_id="primary",retention_rules=[BucketRetentionRules(every_seconds=60*60)])  #create bucket with retention rule

ApiException: (422)
Reason: Unprocessable Entity
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json; charset=utf-8', 'X-Influxdb-Build': 'OSS', 'X-Influxdb-Version': 'v2.7.5', 'X-Platform-Error-Code': 'conflict', 'Date': 'Mon, 19 Feb 2024 13:53:02 GMT', 'Content-Length': '84'})
HTTP response body: {
	"code": "conflict",
	"message": "bucket with name myFirstBucket already exists"
}


Minimum retention time is 1h!

**Cross Check now in the User Interface**

# Write Data

In [ ]:
write_api = client.write_api()

The batch item wasn't processed successfully because: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json; charset=utf-8', 'X-Influxdb-Build': 'OSS', 'X-Influxdb-Version': 'v2.7.5', 'X-Platform-Error-Code': 'invalid', 'Date': 'Mon, 19 Feb 2024 13:49:58 GMT', 'Content-Length': '115'})
HTTP response body: {"code":"invalid","message":"unable to parse 'myMeasurement,mytag=value joint1=1, joint2=2': invalid field format"}

The batch item wasn't processed successfully because: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json; charset=utf-8', 'X-Influxdb-Build': 'OSS', 'X-Influxdb-Version': 'v2.7.5', 'X-Platform-Error-Code': 'invalid', 'Date': 'Mon, 19 Feb 2024 13:50:19 GMT', 'Content-Length': '115'})
HTTP response body: {"code":"invalid","message":"unable to parse 'myMeasurement,mytag=value joint1=1, joint2=2': invalid field format"}

The batch item wasn't processed successfully because: (400)
Re

A bucket is a named location where time series data is stored. All buckets have a retention period, a duration of time that each data point persists. InfluxDB drops all points with timestamps older than the bucket’s retention period. A bucket belongs to an organization.

Syntax:

```<measurement>[,<tag_key>=<tag_value>[,<tag_key>=<tag_value>]] <field_key>=<field_value>[,<field_key>=<field_value>] [<timestamp>]```

In [ ]:
write_api.write("myFirstBucket", "primary", ["myMeasurement,location=coyote_creek water_level=11.4"]) # bucket, organization, measurement: h2o_feet,  location: coyote_creek

measurement: h2o_feet

location: coyote_creek

field: water_level

Add multiple filed by using `,`

**What is a tag?**

In [ ]:
write_api.write("myFirstBucket", "primary", ["myMeasurement,mytag=a_tag_value temperature=2"])  #send data with tag

In [ ]:
write_api.write("myFirstBucket", "primary", ["myMeasurement,mytag=a_tag_value temperature=12"]) #send data with tag

Attention: if sent too fast, they are aggregated!

Create a loop for publishing multiple data-points

In [ ]:
for i in range(1,10):
    write_api.write("myFirstBucket", "primary", ["robotsensor,location=robotcell temperature={0},humidity={1}".format(random.random()*10,random.random())])
    time.sleep(1)

This is already hard to read. So the api provides a build-pattern mechanism.

In [ ]:
p = influxdb_client.Point("robotsensor").tag("location", "robotcell").tag("qos", "2").field("temperature", 20.)
write_api.write(bucket="myFirstBucket", org="primary", record=p)

Fields are data and tags are metadata

## Query

In [ ]:
query_api =client.query_api()

In [ ]:
tables=query_api.query('from(bucket:"myFirstBucket") |> range(start: -10m)')  #last 10 minutes

Here, we get all fields!

In [ ]:
for table in tables:
    print(table)
    for row in table.records:
        print (row.values)

FluxTable() columns: 10, records: 1
{'result': '_result', 'table': 0, '_start': datetime.datetime(2024, 2, 19, 13, 35, 5, 889181, tzinfo=tzutc()), '_stop': datetime.datetime(2024, 2, 19, 13, 45, 5, 889181, tzinfo=tzutc()), '_time': datetime.datetime(2024, 2, 19, 13, 44, 26, 952759, tzinfo=tzutc()), '_value': 20.0, '_field': 'temperature', '_measurement': 'robotsensor', 'location': 'robotcell', 'qos': '2'}


**Might be hard to read: convert that to a pandas**

In [ ]:
query_api.query_data_frame('from(bucket:"myFirstBucket") |> range(start: -20m)')[0]  #last 10 minutes

/home/dinu/Study/Semester-6/Current Trends and Emerging Technologies/venv/lib/python3.11/site-packages/influxdb_client/client/warnings.py:31: MissingPivotFunction: The query doesn't contains the pivot() function.

The result will not be shaped to optimal processing by pandas.DataFrame. Use the pivot() function by:

    from(bucket:"myFirstBucket") |> range(start: -20m) |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")

You can disable this warning by:
    import warnings
    from influxdb_client.client.warnings import MissingPivotFunction

    warnings.simplefilter("ignore", MissingPivotFunction)

For more info see:
    - https://docs.influxdata.com/resources/videos/pivots-in-flux/
    - https://docs.influxdata.com/flux/latest/stdlib/universe/pivot/
    - https://docs.influxdata.com/flux/latest/stdlib/influxdata/influxdb/schema/fieldsascols/

  warnings.warn(message, MissingPivotFunction)


,result,table,_start,_stop,_time,_value,_field,_measurement,location
0,_result,0,2024-02-19 13:25:39.197821+00:00,2024-02-19 13:45:39.197821+00:00,2024-02-19 13:26:44.185761+00:00,0.544757,humidity,robotsensor,robotcell
1,_result,0,2024-02-19 13:25:39.197821+00:00,2024-02-19 13:45:39.197821+00:00,2024-02-19 13:26:45.186627+00:00,0.798996,humidity,robotsensor,robotcell
2,_result,0,2024-02-19 13:25:39.197821+00:00,2024-02-19 13:45:39.197821+00:00,2024-02-19 13:26:46.188060+00:00,0.399321,humidity,robotsensor,robotcell
3,_result,0,2024-02-19 13:25:39.197821+00:00,2024-02-19 13:45:39.197821+00:00,2024-02-19 13:26:47.189257+00:00,0.330867,humidity,robotsensor,robotcell
4,_result,0,2024-02-19 13:25:39.197821+00:00,2024-02-19 13:45:39.197821+00:00,2024-02-19 13:26:48.190149+00:00,0.828383,humidity,robotsensor,robotcell
5,_result,0,2024-02-19 13:25:39.197821+00:00,2024-02-19 13:45:39.197821+00:00,2024-02-19 13:26:49.191433+00:00,0.637324,humidity,robotsensor,robotcell
6,_result,0,2024-02-19 13:25:39.197821+00:00,2024-02-19 13:45:39.197821+00:00,2024-02-19 13:26:50.193550+00:00,0.132214,humidity,robotsensor,robotcell
7,_result,0,2024-02-19 13:25:39.197821+00:00,2024-02-19 13:45:39.197821+00:00,2024-02-19 13:26:51.194741+00:00,0.237691,humidity,robotsensor,robotcell
8,_result,0,2024-02-19 13:25:39.197821+00:00,2024-02-19 13:45:39.197821+00:00,2024-02-19 13:26:52.194095+00:00,0.362900,humidity,robotsensor,robotcell
9,_result,0,2024-02-19 13:25:39.197821+00:00,2024-02-19 13:45:39.197821+00:00,2024-02-19 13:27:55.238935+00:00,0.300612,humidity,robotsensor,robotcell


In [ ]:
import warnings
from influxdb_client.client.warnings import MissingPivotFunction

warnings.simplefilter("ignore", MissingPivotFunction)

We get a warning, that the representation might not be optimal and pivot should be used.

For the examples discussed here, we will stick to the the current setup!

https://docs.influxdata.com/flux/v0.x/stdlib/universe/pivot/


## Task

Publish to a new measurement `robot` two fields `joint1` and `joint2` with arbitrary values.

In [42]:
# write_api.write("myFirstBucket", "primary", ["robot,mytag=value joint1=1, joint2=2"])  #send data with tag
# p = influxdb_client.Point("robot").tag("location", "robotcell").tag("qos", "2").field("joint1", 1).field("joint2", 2)
# write_api.write(bucket="myFirstBucket", org="primary", record=p)
write_api.write("myFirstBucket", "primary", ["robot,location=robotcell joint1=11, joint2=22"])  #send data with tag


## Queries

Often, we want to make a query more `re-useable` and so we can distinguish between params and values!

In [43]:
q = '''
    from(bucket: stringParam)
      |> range(start: -225m, stop: now())
      |> filter(fn: (r) => r._measurement == "myMeasurement")
'''

p = {
    "stringParam": "myFirstBucket",
}



In [44]:
result = query_api.query_data_frame(query=q, params=p)

In [45]:
result[0]

,result,table,_start,_stop,_time,_value,_field,_measurement,location
0,_result,0,2024-02-19 10:12:02.143289+00:00,2024-02-19 13:57:02.143289+00:00,2024-02-19 13:19:58.917314+00:00,11.4,water_level,myMeasurement,coyote_creek
1,_result,0,2024-02-19 10:12:02.143289+00:00,2024-02-19 13:57:02.143289+00:00,2024-02-19 13:21:26.973768+00:00,11.4,water_level,myMeasurement,coyote_creek


The _start and _stop timestamps are the timestamps that corresponds with the range() portion of your query

We see, that if we write multiple messages at exactly the same time, they will be overwritten!

In [46]:
write_api.write("myFirstBucket", "primary", ["myMeasurement,location=coyote_creek water_level=4"],write_precision=WritePrecision.MS)
time.sleep(0.1)
write_api.write("myFirstBucket", "primary", ["myMeasurement,location=coyote_creek water_level=2"]) 
write_api.write("myFirstBucket", "primary", ["myMeasurement,location=coyote_creek water_level=3,temperature=9"]) 
# if we have 2 points that have same data same time, the last one will be kept

`For points that have the same measurement name, tag set, and timestamp, InfluxDB creates a union of the old and new field sets`

In [48]:
tables=query_api.query_data_frame('from(bucket:"myFirstBucket") \
                                  |> range(start: -10m)  \
                                  |> filter(fn: (r) => r._measurement == "myMeasurement") \
                                  |> filter(fn: (r) => r.location == "coyote_creek") \
                                  ')  #last 10 minutes

tables

,result,table,_start,_stop,_time,_value,_field,_measurement,location
0,_result,0,2024-02-19 13:47:14.017875+00:00,2024-02-19 13:57:14.017875+00:00,2024-02-19 13:57:06.495740+00:00,9.0,temperature,myMeasurement,coyote_creek
1,_result,1,2024-02-19 13:47:14.017875+00:00,2024-02-19 13:57:14.017875+00:00,2024-02-19 13:57:06.477000+00:00,4.0,water_level,myMeasurement,coyote_creek
2,_result,1,2024-02-19 13:47:14.017875+00:00,2024-02-19 13:57:14.017875+00:00,2024-02-19 13:57:06.495740+00:00,3.0,water_level,myMeasurement,coyote_creek
3,_result,1,2024-02-19 13:47:14.017875+00:00,2024-02-19 13:57:14.017875+00:00,2024-02-19 13:57:12.481857+00:00,4.0,water_level,myMeasurement,coyote_creek


However, if a tag is different, then we do not have that behaviour!

In [49]:
write_api.write("myFirstBucket", "primary", ["myMeasurement,location=coyote_creek water_level=4"])
write_api.write("myFirstBucket", "primary", ["myMeasurement,location=other water_level=2"]) 

In [50]:
tables=query_api.query_data_frame('from(bucket:"myFirstBucket") |> range(start: -1m)  \
                                  |> filter(fn: (r) => r._measurement == "myMeasurement") \
                                  ')  #last 10 minutes

tables

,result,table,_start,_stop,_time,_value,_field,_measurement,location
0,_result,0,2024-02-19 13:56:17.115470+00:00,2024-02-19 13:57:17.115470+00:00,2024-02-19 13:57:06.495740+00:00,9.0,temperature,myMeasurement,coyote_creek
1,_result,1,2024-02-19 13:56:17.115470+00:00,2024-02-19 13:57:17.115470+00:00,2024-02-19 13:57:06.477000+00:00,4.0,water_level,myMeasurement,coyote_creek
2,_result,1,2024-02-19 13:56:17.115470+00:00,2024-02-19 13:57:17.115470+00:00,2024-02-19 13:57:06.495740+00:00,3.0,water_level,myMeasurement,coyote_creek
3,_result,1,2024-02-19 13:56:17.115470+00:00,2024-02-19 13:57:17.115470+00:00,2024-02-19 13:57:12.481857+00:00,4.0,water_level,myMeasurement,coyote_creek
4,_result,1,2024-02-19 13:56:17.115470+00:00,2024-02-19 13:57:17.115470+00:00,2024-02-19 13:57:15.484158+00:00,4.0,water_level,myMeasurement,coyote_creek
5,_result,2,2024-02-19 13:56:17.115470+00:00,2024-02-19 13:57:17.115470+00:00,2024-02-19 13:57:12.481857+00:00,2.0,water_level,myMeasurement,other
6,_result,2,2024-02-19 13:56:17.115470+00:00,2024-02-19 13:57:17.115470+00:00,2024-02-19 13:57:15.484158+00:00,2.0,water_level,myMeasurement,other


**More Complex Query**

With keep you can limit the number of fields - `keep` is similar to `select` in SQL

In [ ]:
tables=query_api.query_data_frame("""
                        from(bucket:"myFirstBucket") 
                        |> range(start: -2135m) 
                        |> filter(fn: (r) => r._measurement == "myMeasurement")
                        |> filter(fn: (r) => r._field == "temperature")
                        |> keep(columns:["_value"])
                        """)  

tables

In [ ]:
tables=query_api.query_data_frame("""
                        from(bucket:"myFirstBucket") 
                        |> range(start: -215m) 
                        |> filter(fn: (r) => r._measurement == "robotsensor")
                        |> filter(fn: (r) => r._field == "temperature")
                        """)  

Joins are also existing!

In [ ]:
tables=query_api.query_data_frame("""
                        mymeasure=from(bucket:"myFirstBucket") 
                        |> range(start: -215m) 
                        |> filter(fn: (r) => r._measurement == "myMeasurement")
                        |> filter(fn: (r) => r._field == "temperature")

                        robotsensor=from(bucket:"myFirstBucket") 
                        |> range(start: -215m) 
                        |> filter(fn: (r) => r._measurement == "robotsensor")
                        |> filter(fn: (r) => r._field == "temperature")
                        
                        
                        join(tables: {mymeasure:mymeasure, robotsensor:robotsensor}, on: ["_field"], method: "inner")
                        """)  

tables[0]

Filtering for tags is also possible!

In [ ]:
tables=query_api.query_data_frame("""
                        from(bucket:"myFirstBucket") 
                        |> range(start: -2135m) 
                        |> filter(fn: (r) => r._measurement == "myMeasurement")
                        |> filter(fn: (r) => r.mytag == "a_tag_value")
                        """)  

tables

We can also create bins!

In [ ]:
my_query="""from(bucket:"myFirstBucket") 
                        |> range(start: -2135m) 
                        |> filter(fn: (r) => r._measurement == "myMeasurement")
                        |> filter(fn: (r) => r._field == "temperature")
                        |> filter(fn: (r) => r.mytag == "a_tag_value")
                        |> histogram(
                            column: "_value",
                            upperBoundColumn: "le",
                            countColumn: "_value",
                            bins: [100.0, 200.0, 300.0, 400.0],
                          )
                        """

In [ ]:
query_api.query_data_frame(org="primary",query=my_query)

cumulative distribution - `_value`

**Moving Average**

In [ ]:
tables=query_api.query_data_frame("""from(bucket:"myFirstBucket") 
                        |> range(start: -4435m) 
                        |> filter(fn: (r) => r._measurement == "myMeasurement")
                        |> filter(fn: (r) => r._field == "temperature")
                        |> movingAverage(n:5)
                        """)

tables[3]

movingAverage() calculates the mean of non-null values using the current value and n - 1 previous values in the _values column.

In [ ]:
write_api.write("myFirstBucket", "primary", ["myMeasurement,mytag=a_tag_value temperature=4"])

## Super-Simple Exercises

What does `fill` do?

What does `group` do?

In [ ]:
# example with group
write_api.write("myFirstBucket", "primary", ["myMeasurement,location=coyote_creek temperature=12"]) 


In [ ]:
write_api.write("myFirstBucket", "primary", ["myMeasurement,location=other temperature=15"]) 


In [ ]:
write_api.write("myFirstBucket", "primary", ["myMeasurement,location=coyote_creek temperature=17"]) 


In [ ]:
write_api.write("myFirstBucket", "primary", ["myMeasurement,location=coyote_creek temperature=9"]) 

In [ ]:
tables=query_api.query_data_frame("""from(bucket:"myFirstBucket") 
                        |> range(start: -5m) 
                        |> filter(fn: (r) => r._measurement == "myMeasurement")
                        |> filter(fn: (r) => r._field == "temperature")
                        """)

tables #raw data

In [ ]:
tables=query_api.query_data_frame("""from(bucket:"myFirstBucket") 
                        |> range(start: -5m) 
                        |> filter(fn: (r) => r._measurement == "myMeasurement")
                        |> filter(fn: (r) => r._field == "temperature")
                        |> group(columns: ["location"])
                        |> mean()
                        """)

tables

In [ ]:
#now, lets make a group by - having construct

tables=query_api.query_data_frame("""from(bucket:"myFirstBucket") 
                        |> range(start: -8m) 
                        |> filter(fn: (r) => r._measurement == "myMeasurement")
                        |> filter(fn: (r) => r._field == "temperature")
                        |> group(columns: ["location"])
                        |> count()
                        |> filter(fn: (r) => r._value>1)
                        """)

tables

## Query Exercises

Write a Query that gets only the mean of the field `temperature`, ignore data older than 5 minutes. The aggregtion should ignore tags (drop)!

In [55]:
mean_last_5m = query_api.query_data_frame("""
    from (bucket:"myFirstBucket")
        |> range(start: -5m)
        |> filter(fn: (r) => r._measurement == "myMeasurement")
        |> filter(fn: (r) => r._field == "temperature")
        |> drop(columns:["my_tag"])
        |> mean()
""")

In [56]:
mean_last_5m

""


In [ ]:
tables[0]

Write a query that gets 1-minute windows (`aggregateWindow(every: 1m, fn: mean)`) of the field `temperature`, ignore windows that have no values (hint: use `filter` with `exists r._value`)!

Find the `temperature` peak of the last 60 minutes. Hint: use `max`

Find the three highest `temperatures`. Hint use `top`

Find the `water_level` value of the value with the highest `temperatures` (hint: use `join` with  `_time` as join attribute).

In [ ]:
write_api.write("myFirstBucket", "primary", ["myMeasurement,mytag=a_tag_value temperature=49,water_level=12"])

Convert the temperature to Fahreinheit (hint use ` |> map(fn: (r) => ({ r with _value: (r._value*9./5.)+32.}))`:)

Flux also supports `high-level` functions to create e.g. histograms

# Exercise

**A**

Create two MQTT Clients: one publisher and one subscriber.
The publiblisher should publish values in an infinite loop. The subscriber subscribes to that values and stores them to InfluxDB.


**B**

Design: how to treate different topics? How should the topic structure be represented in MQTT? Look e.g.h [here](https://github.com/mhaas/mqtt-to-influxdb-forwarder)

Try to implement it!


# Grafana

Start with the following query

```
from(bucket: "myFirstBucket")
  |> range(start: -1115m)
  |> filter(fn: (r) =>
    r._measurement == "myMeasurement" and
    r._field == "temperature"
  )
```

add multiple queries to Grafana!

Create a bar gauge showing the max temperature values by each tag `location`. Therefore, use `|> filter(fn: (r) => exists r.location)` in the query

Use Grafanas time range instead of range: `range(start: v.timeRangeStart, stop:v.timeRangeStop)`